In [1]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.metrics import confusion_matrix,accuracy_score

C:\Users\tusha\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
amaz_data=pd.read_csv('amazon_cells_labelled.txt',sep='\t',header=None)

In [3]:
imbd_data=pd.read_csv('imdb_labelled.txt',sep='\t',header=None)

In [4]:
yelp_data=pd.read_csv('yelp_labelled.txt',sep='\t',header=None)

In [5]:
df=pd.DataFrame()

In [6]:
df=df.append(amaz_data)
df=df.append(imbd_data)
df=df.append(yelp_data)

In [7]:
df.head()

,0,1
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [9]:
df.columns=['reviews','sentiment']

In [10]:
df.head()

,reviews,sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [11]:
def textprocessing(data):
    tokens=[]
    for i in range(len(data)):
        tokens.append(word_tokenize(df['reviews'].iloc[i].lower()))
    
    eng_stopwords=stopwords.words('english')
    eng_stopwords.extend([',','?','!','@','.','-','&'])
    words=[]
    for token in tokens:
        list1=[]
        for i in range(len(token)):
            if token[i] not in eng_stopwords:
                list1.append(token[i])
        words.append(list1)
    
    wnet=WordNetLemmatizer()
    for i in range(len(words)):
        for j in range(len(words[i])):
            words[i][j]=wnet.lemmatize(words[i][j],pos='v')
            
    for i in range(len(words)):
        words[i]=' '.join(words[i])
    
    return words
        

In [12]:
wordlist=textprocessing(df)

In [13]:
wordlist[100]

'integrate seamlessly motorola razr phone'

In [14]:
tfidf=TfidfVectorizer()

In [16]:
vector=tfidf.fit_transform(wordlist)

In [17]:
vector[0]

<1x4372 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [18]:
vector=vector.toarray()

In [20]:
vector[0:10]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [21]:
y=df['sentiment']

In [22]:
x_train,x_test,y_train,y_test=train_test_split(vector,y,test_size=0.25)

In [23]:
nb=GaussianNB()

In [25]:
nb.fit(x_train,y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [26]:
y_pred=nb.predict(x_test)

In [27]:
accuracy_score(y_test,y_pred)

0.7336244541484717

In [28]:
mb=MultinomialNB()

In [29]:
mb.fit(x_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [30]:
y_predict=mb.predict(x_test)

In [31]:
accuracy_score(y_test,y_predict)

0.8253275109170306

In [32]:
reg=LogisticRegression()

In [33]:
reg.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [34]:
y_pre=reg.predict(x_test)

In [35]:
accuracy_score(y_test,y_pre)

0.8078602620087336

In [36]:
new_rev = "Hrithik and Tiger both are genetically gifted and they have worked hard on their bodies. Offcourse they had enough funds available to do that since they were born. But these are not enough to make a movie good, same old cliche, same old stunts. Seen thousand times in the past. This is an era when you can't fool the audience with nobrainers like these, action movies are good when action is supported with a nice plot or storyline. Nothing is in the movie, the movie will do well on the box office initially because of the heavy marketing and the appeal of the lead actors but it won't go further. A third grade movie which tries hard to be a poor copy of famous Hollywood action movies. Only villagers and autowallahs will love this."

In [38]:
test_df=pd.DataFrame({"reviews":[new_rev]})

In [39]:
test_wordlist=textprocessing(test_df)

In [40]:
test_vector=tfidf.transform(test_wordlist)

In [42]:
test_matrix=test_vector.toarray()

In [44]:
mb.predict(test_matrix)

array([0], dtype=int64)

In [45]:
import pickle as pkl

In [46]:
file=open('tfidf.pkl','wb')
pkl.dump(tfidf,file)
file.close()

file=open('nbb.pkl','wb')
pkl.dump(mb,file)
file.close()
